In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import numpy as np
import sys
import matplotlib.pyplot as plt
import torch
import cv2
sys.path.append('..')
sys.path.append('../visualization')
sys.path.append('../acd/util')
sys.path.append('../acd/scores')
sys.path.append('../acd/agglomeration')

import tiling_2d as tiling
import agg_2d as agg
import cd
import score_funcs
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("device", device)
# get dataset
import pickle as pkl
from dsets.imagenet import dset
#----
# for idx, t in enumerate(imnet_dict[9]):
#     print("idx", idx, "imnet_dict[9]",type(t), t.shape ,t)
#----
# get model
from torchvision import models
model = models.vgg16(pretrained=True).to(device).eval()
model_type='vgg' # alexnet, vgg

In [ ]:
# hyperparameters
num_iters = 5 # number of iterations to agglomerate before merging remaning blobs (fig uses 4)
percentile_include = 95 # values above this percentile will be added at each iteration (fig uses 95)
method = 'cd' # method to rank importance ('cd' works best, 'build_up' or 'occlusion' are simplest)
sweep_dim = 14 # importances are calculated by blocks of sweep_dim x sweep_dim (14 yields good results for imagenet)
for key_num in [-1]: 
    im_orig = cv2.imread('/Users/yezheng/github/ExtremeNet/16004479832_a748d55f21_k.jpg')
    print("im_orig", im_orig)
    lab_pred = np.argmax(dset.pred_ims(model, np.copy(im_orig)))
    print("key_num",key_num,"lab_pred")
    lists = agg.agglomerate(model, dset.pred_ims, percentile_include, method, sweep_dim, im_orig, 
                            lab_pred, num_iters=num_iters, im_torch=im_torch, model_type=model_type, device = device)        
    print("key_num",key_num,"agg.agglomerate")

    # visualize
    plt.figure(figsize=(12, 5), facecolor='white', dpi=100)
    rows = 3      
    num_ims = len(lists['scores_list'])
    import viz_2d as viz
#     # original plots
#     ind, labs = viz.visualize_original_preds(im_orig, lab_num_correct, 
#                                              lists['comp_scores_raw_list'], lists['scores_orig_raw'],
#                                              subplot_rows=rows, dset=dset, key_num  =key_num)

    # comp plots
    viz.visualize_ims_list(lists['comps_list'],
                          title='Chosen blobs',
                          subplot_row=1, subplot_rows=rows, colorbar=False, im_orig=im_orig, plot_overlay=True, key_num  =key_num)

    # dict plots
    viz.visualize_dict_list_top(lists['comp_scores_raw_list'], method,
                           subplot_row=2, subplot_rows=rows, ind=ind, labs=labs, use_orig_top=True, key_num  =key_num)